# Project 1: Vacuum Cleaner Agent
## Author: Mia Salvati, MWF 9:00 AM to 9:50 AM



## Assignment:
Create vacuum cleaner environment which clearly separates the sensor, actuators and environment from the agent portion of the code (Look at the blind dog problem, remember a hungry dog is like a vacuum cleaner).

The environment is a 10x10 square room with a given probalibity of dirt appearing in any square specificed.  The robot vacuum starts in a random location and cleans for a given number of time steps. 

Create the following types of agents and measure the performance of each.  Explain your results.

- Simple (deterministic) reflex agent (just sense, act, sense, act), the agent has a dirt sensor and a bump sensor. It can clean, turn (left or right), and move forward.

- Simple (random) reflex agent, as above but can take a random action.

- A reflex agent which is deterministic but can remember the last 4 sensor inputs and last 4 actions.


Introduction:

In this notebook, I am going to design 3 agents. All of them will have two sensors. One will be a dirt sensor, which will alert the vacuum that the square underneath it is dirty and has to be cleaned. The cleaning of the square under it takes priority before moving the agent. The second type of sensor is a bump sensor. This alerts the vacuum with a "bump" if it runs into the wall.

The first agent will be a simple reflex agent, which will only be able to move forward and turn one way when it is alerted that it has bumped into the wall. The second one will be a simple random reflex agent, which is the same as the one above in most ways. However, it will also have the chance to take a random action. Unlike the first simple reflex agent though, it will be able to move both left or right, instead of only one direction. The third agent is a deterministic reflex agent. It remembers the last 4 sensor inputs and the last 4 actions given to it. Based on the sequence of the last 4 inputs and last 4 actions, it is coded to take a specific action. This is using an "if this, then that" action approach to its methods.

Discussion of Solution:

I am using modified code, based on the Blind Dog agent example code given by the AIMA github and Chapter 2 of the AIMA textbook. I have implemented this in a few ways.

With the first simple reflex agent, I took the final blind dog code that utilized the 2D space and modified it a bit. I made it so that it can only turn one way. This agent also only turns if it bumps into things. If it does not bump into things, all it is permitted to do is go down. Cleaning is its first priority though, so it checks first to see if there is dirt underneath it. This is modified from the "eat" and "drink" actions given by the original code.

The second code is very similar to the blind dog code given. This code uses a random number generator to decide on the vacuum's actions in terms of turning. The cleaning function has been modified again, as the most important thing for this agent to do is clean.

The third code will be designed using the idea of a space filling curve. Since the agent remembers the last 4 actions that it takes and the last 4 sensors inputs it got, we will use that memory store to our advantage. First, the agent will always go one way (down or up). When it hits a bump, it will turn randomly, move one space forward, and then repeat the turn. Then it will go in whatever direction it is facing. In theory, this will eventually clean the entire room and operate better than the simple random reflex agent.

A note: QuantumRandom was not working, so I am using the Python Random Number Generator built in to generate numbers.

In [19]:
!pip install ipythonblocks

This is the room and agent. I have only modified it slightly, by adding a turn function.

In [20]:
import sys
sys.path.append('/home/nbuser/library/')
# This is used to include files which are in the same folder.

from utils import *
from agents import * 


# agents.py must be in the same folder as the notebook
# utils.py must be in the same folder as the notebook

class GraphicRoom(GraphicEnvironment):
    def __init__(self, width=10, height=10, boundary=True, color={}, display=False):
        super().__init__(width, height, boundary, color, display)
        
    def thing_classes(self):
        return [Wall, Dirt, Bump, ReflexVacuumAgent, RandomVacuumAgent,
                TableDrivenVacuumAgent, ModelBasedVacuumAgent]    
        
    def percept(self, agent):
        '''return a list of things that are in our agent's location'''
        things = self.list_things_at(agent.location)
        return things
    
    def execute_action(self, agent, action):
        '''changes the state of the environment based on what the agent does.'''
        # This bump was from the last move, so remove it
        items = self.list_things_at(agent.location, tclass=Bump)
        if len(items) != 0:
            self.delete_thing(items[0])
            
        if action == 'turnright':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.R)
        elif action == 'turnleft':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.L)
        elif action == 'moveforward':
            loc = copy.deepcopy(agent.location) # find out the target location
            if agent.direction.direction == Direction.R:
                loc[0] += 1
            elif agent.direction.direction == Direction.L:
                loc[0] -= 1
            elif agent.direction.direction == Direction.D:
                loc[1] += 1
            elif agent.direction.direction == Direction.U:
                loc[1] -= 1
            if self.is_inbounds(loc):# move only if the target is a valid location
                print('{} decided to move {}wards at location: {}'.format(str(agent)[1:-1], agent.direction.direction, agent.location))
                agent.moveforward()
                agent.location = loc  # We can move forward so update it...the vacuum doesn't know anything
            else:
                print('{} decided to move {}wards at location: {}, but couldn\'t'.format(str(agent)[1:-1], agent.direction.direction, agent.location))
                self.add_thing(Bump(),agent.location)
                print(self.list_things_at(agent.location))
                agent.moveforward(False)
               
        elif action == "clean":
            items = self.list_things_at(agent.location, tclass=Dirt)
            if len(items) != 0:
                if agent.clean(items[0]):
                    print('{} cleaned {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0])
        
                    
    def is_done(self):
       
        return False

class Dirt(Thing):
    pass

class Bump(Thing):
    pass

class Vacuum(Agent):
    
    direction = Direction("down")
        
    def clean(self, thing):
        print("Vacuum Cleaned Location {}.".format(self.location))
            
    def moveforward(self, success=True):
        print("Vacuum Moved Forward {}.".format( self.location))
    
    def turn(self, d):
        self.direction = self.direction + d

def program(percepts):
    print(percepts)
    return('moveforward')

Testing Design:

In this, I will be using the same type of tests for all three of the agent programs. The performance will be measured by how much dirt is cleaned in the room versus how much dirt there was to begin with. Each agent will be tested with 10% of the room dirty, 20% of the room dirty, and 30% of the room dirty. Each of these dirt probabilities will be tested 200 times each and averaged, to get an overall view of what each agent can do versus focusing on a limited amount of trial runs. In each room, the vacuum will be allowed to run 500 times. This should get a good overall performance of each agent. I will spare the graphics of each of these tests, as this notebook is long enough. However, the graphics are tested for each one below. They all have 10 moves each on their graphical test. 

Expected Results:

In this, I expect the first simple reflex agent program to do poorly. It is designed to only move forward until it bumps into a wall. Then it turns one way and repeats the process. In theory this will only hug the sides of the wall once it bumps that first time. Any dirt that isn't on the edge of the room or on its immediate path when starting off will not get cleaned, making it a very inefficient vacuum. With the second vacuum program, I expect it to do alright. It will do better than the first agent, because it has the ability to take a random action instead of having all of its actions predetermined, making it so that it actually has the ability to clean the entire room. However, it is not guaranteed to clean the entire room, and the amount of time and actions it will take to clean the roomo is all left up to chance and the random number generatore within Python itself. Since it is all chance though, this is the agent that is the best to compare the results of the other two to. The third agent program should have the best performance in theory. It is designed to get every space in the room with enough time. It remembers the last 4 actions taken and determines what to do based on sequences of actions and sensors thought of prior. Overall, this should beat the random agent program's performance.

In [21]:
def simplereflexprogram(percepts):
    
    for p in percepts: 
        if isinstance(p, Dirt): #if dirty, clean it 
            return 'clean'
    
        if isinstance(p, Bump):
            return 'turnright'
    return 'moveforward'
    

I will test the simple reflex program and show the graphics in the program below. 

In [22]:
from random import randint

room=GraphicRoom(12,12, color={'Vacuum': (200,0,0), 'Dirt': (0, 200, 200), 'Wall': (0, 0, 0),'Bump':(200,0,200)})
#the wall is in 0 and location 11

location = [randint(1,10),randint(8,10)]
vac = Vacuum(simplereflexprogram)



room.add_thing(vac,location)
room.add_thing(Dirt(),[randint(1,10),randint(1,10)])
room.add_thing(Dirt(),[randint(1,10),randint(1,10)])
room.add_thing(Dirt(),[randint(1,10),randint(1,10)])

room.reveal()
room.run(10)

,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,


,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,


Vacuum decided to move downwards at location: [8, 9]
Vacuum Moved Forward [8, 9].


,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,


Vacuum decided to move downwards at location: [8, 10], but couldn't
[<Vacuum>, <Bump>]
Vacuum Moved Forward [8, 10].


...

,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,


Vacuum decided to move leftwards at location: [3, 10]
Vacuum Moved Forward [3, 10].


,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,


Vacuum decided to move leftwards at location: [2, 10]
Vacuum Moved Forward [2, 10].


,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,


In [23]:
def randomreflexprogram(percepts):
    
    for p in percepts: 
        if isinstance(p, Dirt): #if dirty, clean
            return 'clean'
       
        if isinstance(p, Bump):
            choice = random.choice((1,2))
            
        else:
            choice = random.choice((1,2,3,4)) # 1-right, 2-left, others-forward
    
        if choice == 1:
            return 'turnright'
        
        elif choice == 2:
            return 'turnleft'
        
    return 'moveforward'

Below, I will test the random reflex program with graphics. 

In [24]:
from random import randint

room=GraphicRoom(12,12, color={'Vacuum': (200,0,0), 'Dirt': (0, 200, 200), 'Wall': (0, 0, 0),'Bump':(200,0,200)})
#the wall is in 0 and location 11

location = [randint(1,10),randint(8,10)]
vac = Vacuum(randomreflexprogram)



room.add_thing(vac,location)
room.add_thing(Dirt(),[randint(1,10),randint(1,10)])
room.add_thing(Dirt(),[randint(1,10),randint(1,10)])
room.add_thing(Dirt(),[randint(1,10),randint(1,10)])

room.reveal()
room.run(10)

,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,


,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,


Vacuum decided to turnleft at location: [1, 10]


,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,


Vacuum decided to turnright at location: [1, 10]


...

,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,


Vacuum decided to turnright at location: [1, 6]


,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,


Vacuum decided to turnright at location: [1, 6]


,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,


In [30]:
sequence = [0, 0, 0, 0]

#this is a list to keep track of movement sensor inputs. It goes from sequence[0] being the oldest move to sequence[3] being the newest move.
#in this sequence, 1 will equal turn right, 2 will equal turn left, 3 will equal bump, 4 will equal move forward

def memorybasedprogram(percepts):

    
    for p in percepts:
        if isinstance(p, Dirt): #cleaning takes priority over all
            return 'clean'
        
    if (sequence == [4, 3, 1, 4]):
        sequence.append(1) #add turn right to the memory
        sequence.pop(0) #get rid of the old memory
        return 'turnright'
    
    if (sequence == [4, 3, 2, 4]):
        sequence.append(2) #add turn left to the memory
        sequence.pop(0) #get rid of the old memory
        return 'turnleft'
    
    for p in percepts:
        if isinstance(p, Bump): #if a bump is triggered
            sequence.append(3) #add bump to the memory
            sequence.pop(0) #get rid of the old memory
            choice = random.choice((1,2)) #if you have to turn, turn left or right, this is right after bump only.
            sequence.append(choice) #add this turn of left or right to the memory
            sequence.pop(0) #get rid of oldest memory
            
            if choice == 1:
                return 'turnright'
            elif choice == 2:
                return 'turnleft'
    
    sequence.append(4) #add the action of moving forward into the sequence
    sequence.pop(0) #getting rid of the old sequence
    return'moveforward'
        
        

Here, I will test the memory based agent program with graphics. 

In [31]:
from random import randint

room=GraphicRoom(12,12, color={'Vacuum': (200,0,0), 'Dirt': (0, 200, 200), 'Wall': (0, 0, 0),'Bump':(200,0,200)})
#the wall is in 0 and location 11

location = [randint(1,10),randint(8,10)]
vac = Vacuum(memorybasedprogram)



room.add_thing(vac,location)
room.add_thing(Dirt(),[randint(1,10),randint(1,10)])
room.add_thing(Dirt(),[randint(1,10),randint(1,10)])
room.add_thing(Dirt(),[randint(1,10),randint(1,10)])

room.reveal()
room.run(10)

,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,


,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,


Vacuum decided to move downwards at location: [2, 8]
Vacuum Moved Forward [2, 8].


,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,


Vacuum decided to move downwards at location: [2, 9]
Vacuum Moved Forward [2, 9].


...

,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,


Vacuum decided to move upwards at location: [1, 8]
Vacuum Moved Forward [1, 8].


,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,


Vacuum decided to move upwards at location: [1, 7]
Vacuum Moved Forward [1, 7].


,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,
,,,,,,,,,,,


Actual Testing:

Here is the room that is graphics free. This will save space on the notebook.

In [68]:
class NoGraphicsRoom(XYEnvironment):
    
    def thing_classes(self): #everything under here is the same as the room I had prior, minus graphics
        return [Wall, Dirt, Bump, ReflexVacuumAgent, RandomVacuumAgent,
                TableDrivenVacuumAgent, ModelBasedVacuumAgent]    
        
    def percept(self, agent):
        '''return a list of things that are in our agent's location'''
        things = self.list_things_at(agent.location)
        return things
    
    def execute_action(self, agent, action):
        '''changes the state of the environment based on what the agent does.'''
        # This bump was from the last move, so remove it
        items = self.list_things_at(agent.location, tclass=Bump)
        if len(items) != 0:
            self.delete_thing(items[0])
            
        if action == 'turnright':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.R)
        elif action == 'turnleft':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.L)
        elif action == 'moveforward':
            loc = copy.deepcopy(agent.location) # find out the target location
            if agent.direction.direction == Direction.R:
                loc[0] += 1
            elif agent.direction.direction == Direction.L:
                loc[0] -= 1
            elif agent.direction.direction == Direction.D:
                loc[1] += 1
            elif agent.direction.direction == Direction.U:
                loc[1] -= 1
            if self.is_inbounds(loc):# move only if the target is a valid location
                print('{} decided to move {}wards at location: {}'.format(str(agent)[1:-1], agent.direction.direction, agent.location))
                agent.moveforward()
                agent.location = loc  # We can move forward so update it...the vacuum doesn't know anything
            else:
                print('{} decided to move {}wards at location: {}, but couldn\'t'.format(str(agent)[1:-1], agent.direction.direction, agent.location))
                self.add_thing(Bump(),agent.location)
                print(self.list_things_at(agent.location))
                agent.moveforward(False)
               
        elif action == "clean":
            items = self.list_things_at(agent.location, tclass=Dirt)
            if len(items) != 0:
                if agent.clean(items[0]):
                    print('{} cleaned {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0])
        
                    
    def is_done(self):
       
        return False
    
    def dirtcounter(self): #amount of dirt total
        counter = 0
        for x in (self.things):
            counter = counter + 1
        return counter-1
            
    def movecounter(self): #getting the number of moves
        return self.moves
        
   
class Dirt(Thing):
    pass

class Bump(Thing):
    pass

class Vacuum(Agent):
    
    direction = Direction("down")
        
    def clean(self, thing):
        print("Vacuum Cleaned Location {}.".format(self.location))
            
    def moveforward(self, success=True):
        print("Vacuum Moved Forward {}.".format( self.location))
    
    def turn(self, d):
        self.direction = self.direction + d

def program(percepts):
    print(percepts)
    return('moveforward')

Discussion of Results:

In the end, it was correct that the random reflex agent program and memory agent outperformed the simple reflex. The random and memory seem to be on par though. That strikes me as weird. However, I don't know how to fix the problem of the vacuum cleaning a square multiple times, as seen in some of the runs. It didn't do that in more limited tests with graphics. I know that the almost or perfect 1 of these testes are false. 

Conclusion:

To be honest, I ended up doing this project multiple times, because I didn't understand it. If you take a peek into notebooks prior, there are multiple copies of the assignment, each with different ideas. I thought designing other agents was the idea, which is why the word "program" is stressed so much in my writeup. That did not work, and I would not do that in the future to try and optimize performance. I also don't think that expanding the memory would be best, as four moves seems sufficient. I would attempt to fix the multiple same movements in the code that didn't show up in other, more limited runs of the code. I overthought this assignment, way too much. I would just focus on the programs themselves at times and maybe add more sensors, instead of trying to build more agents. 